<a href="https://colab.research.google.com/github/gts0025/test_projects.py/blob/main/titanic_random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.



In [9]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

titanic_path = "/content/test.csv"

print('Data source import complete.')


Data source import complete.


In [10]:

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#ai stuff
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder as le



#loading files
train_data = pd.read_csv("/content/train.csv")
f_test_data = pd.read_csv("/content/test.csv")
final_ids =  f_test_data["PassengerId"]



# cleaning and encodding really is another level of torture :)
def clean_data(data):
    data["Sex"] = data["Sex"].map({"male":0,"female":1})

    #irelevant column:
    if "Name" in data.columns:
        data = data.drop("Name", axis=1)

    data.Cabin = data.Cabin.bfill().astype(str)
    data.Ticket = data.Ticket.bfill().astype(str)
    data.Embarked = data.Embarked.bfill().astype(str)
    data["Fare"] = data["Fare"].fillna(data["Fare"].median()).astype(float)
    data["Age"] = data["Age"].fillna(data["Age"].median()).astype(int)

    #simple binary encodding

    return data

#encoders:
Ticket_encoder = le()
Cabin_encoder = le()
Embarked_encoder = le()


def encode_data(data):
    data["Ticket"] = Ticket_encoder.transform(data["Ticket"])
    data["Cabin"] = Cabin_encoder.transform(data["Cabin"])
    data["Embarked"] = Embarked_encoder.transform(data["Embarked"])
    return data



train_data = clean_data(train_data)
f_test_data = clean_data(f_test_data)

#train_data.isna().sum()
#f_test_data.isna().sum()

f_test_data.tail()
#print("Unexpected Sex values:", train_data["Sex"].unique())

#fitting:
encodding_train = train_data.drop("Survived",axis = 1)
encodding_data = pd.concat([encodding_train, f_test_data], axis=0, ignore_index=True)


Ticket_encoder.fit(encodding_data["Ticket"])
Cabin_encoder.fit(encodding_data["Cabin"])
Embarked_encoder.fit(encodding_data["Embarked"])
encodding_data = encode_data(encodding_data)

#encodding_data.head()
print("Unexpected Sex values:", train_data["Sex"].unique())
print("Unexpected Sex values:", f_test_data["Sex"].unique())


train_data = encode_data(train_data)
f_test_data = encode_data(f_test_data)


Unexpected Sex values: [0 1]
Unexpected Sex values: [0 1]


In [11]:
y = train_data["Survived"]
x = train_data.drop("Survived",axis = 1)


x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.30, random_state=90)



model = RandomForestClassifier(random_state = 1)
model.fit(x_train,y_train)
predicted = model.predict(x_test)


print("score:",accuracy_score(y_test,predicted,))
print("report:",classification_report(y_test,predicted))


score: 0.8283582089552238
report:               precision    recall  f1-score   support

           0       0.85      0.89      0.87       171
           1       0.79      0.72      0.75        97

    accuracy                           0.83       268
   macro avg       0.82      0.81      0.81       268
weighted avg       0.83      0.83      0.83       268



In [12]:
#prepare final_test:

model.fit(x,y)

f_y = model.predict(f_test_data)
final = pd.concat([f_test_data["PassengerId"], pd.Series(f_y, name="Survived")], axis=1)
final.to_csv("submission.csv", index=False)
final.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
